In [1]:
import numpy as np
import sympy as sp

In [2]:
# function to calculate number of occurrence of a pattern in a sub-config including overlapping ones and circular occurrences
def countOccurrencesCircular(string, substring):
    cnt = 0
    x = string + string
    m = len(substring)
    for i in range(len(string)):
        if substring == x[i: i + m]:
            cnt += 1
    return cnt


In [3]:
# function to calculate number of occurrence of a pattern in a sub-config including overlapping ones
def countOccurrences(sub, string):
    count = start = 0
    while True:
        start = string.find(sub, start) + 1
        if start > 0:
            count+=1
        else:
            return count


In [4]:
# function to calculate probability of conversion of sub-config A to B
def calcProp(A, B):
    a = len(A)
    b = len(B)
    if (a != b or A[0] != B[0] or A[a - 1] != B[b - 1]):
        return 0
    r, p = sp.symbols("r p")
    r = 1
    for i in range(1, a - 1):    # ignoring first and last cell
        if (A[i] == B[i]):    # must remain on its strategy
            if (A[i - 1] == A[i] or A[i] == A[i + 1]):    # checking if the ith cell can flip
                r *= (1 - p)
            else:
                r *= 1
        else:
            if (A[i - 1] == A[i] or A[i] == A[i + 1]):
                r *= p
            else:
                return 0
    return r


In [5]:
def generateAllBinaryStrings(length):
    arr = [""] * 2 ** length
    form = '#0' + str(length + 2) + 'b'
    for i in range(2 ** length):
        arr[i] = format(i, form)[2:]
    return np.array(arr)


In [6]:
# function to calculate the expected difference of number of a list of patterns in a sub-config
def calcExpectedDifference(patterns, sub_config):
    k = len(sub_config) - 2
    n = 2 ** k
    binary_strings = generateAllBinaryStrings(k)
    number_of_patterns = patterns.shape[0]
    expected_differences = [0] * number_of_patterns
    for j in range(number_of_patterns):
        #
        total_expectation = 0
        current_number_of_occurrence = countOccurrences(patterns[j], sub_config[1:-1])
        for i in range(n):
            total_expectation += calcProp(sub_config, sub_config[0] + binary_strings[i] + sub_config[-1]) * (countOccurrences(patterns[j], binary_strings[i]) - current_number_of_occurrence)
        #
        expected_differences[j] = total_expectation
    return np.array(expected_differences)


In [37]:
#
def getAllDifferencesOnExpectationWithGivenPatterns(patterns, pattern_length):
    m = patterns.shape[0]
    #check if all patterns have same length
    for i in range(m):
        if (len(patterns[i]) != pattern_length):
            raise Exception("All patterns should have length " + str(pattern_length) + ", but pattern " + str(i) + " has length of " + str(len(patterns[i])) + ".")

    # binary sub-config length should be equal with 2 * pattern_length + 1
    k = 2 * pattern_length + 1
    n = 2 ** k
    binary_sub_configs = generateAllBinaryStrings(k)
    diffs = [ [0] * m for i in range(n)]
    for j in range(n):
        expectations_of_current_sub_config = calcExpectedDifference(patterns, binary_sub_configs[j])
        diffs[j] = expectations_of_current_sub_config
    
    return np.array(diffs).T


In [39]:
pats = np.array(["000", "001", "011", "100", "110", "111"])
x = getAllDifferencesOnExpectationWithGivenPatterns(pats, 3)

In [43]:
x

array([[-3*p**5 - 15*p**4*(1 - p) - 30*p**3*(1 - p)**2 - 27*p**2*(1 - p)**3 - 9*p*(1 - p)**4,
        -3*p**5 - 15*p**4*(1 - p) - 30*p**3*(1 - p)**2 - 27*p**2*(1 - p)**3 - 9*p*(1 - p)**4,
        -2*p**4 - 8*p**3*(1 - p) - 12*p**2*(1 - p)**2 - 6*p*(1 - p)**3,
        -2*p**5 - 10*p**4*(1 - p) - 19*p**3*(1 - p)**2 - 16*p**2*(1 - p)**3 - 5*p*(1 - p)**4,
        -p**4 - 4*p**3*(1 - p) - 6*p**2*(1 - p)**2 - 3*p*(1 - p)**3,
        -p**3 - 3*p**2*(1 - p) - 3*p*(1 - p)**2,
        -p**5 - 4*p**4*(1 - p) - 7*p**3*(1 - p)**2 - 6*p**2*(1 - p)**3 - 2*p*(1 - p)**4,
        -p**5 - 4*p**4*(1 - p) - 7*p**3*(1 - p)**2 - 6*p**2*(1 - p)**3 - 2*p*(1 - p)**4,
        0, 0, 0, 0,
        2*p**4*(1 - p) + 5*p**3*(1 - p)**2 + 4*p**2*(1 - p)**3 + p*(1 - p)**4,
        p**4 + 3*p**3*(1 - p) + 3*p**2*(1 - p)**2 + p*(1 - p)**3,
        p**5 + 3*p**4*(1 - p) + 4*p**3*(1 - p)**2 + 3*p**2*(1 - p)**3 + p*(1 - p)**4,
        p**5 + 3*p**4*(1 - p) + 4*p**3*(1 - p)**2 + 3*p**2*(1 - p)**3 + p*(1 - p)**4,
        -p**4